### Processamento NDVI Suzano 11.603 polígonos de restauração
### Amostras de referência de pastagem e floresta
#### Criado por José Guilherme Fronza em 31/03/2022
#### NDVI Sentinel-2 TOA 2016-2021 - Semestral
#### Processado no Google Earth Engine e limpo/tratado neste Notebook
## The Nature Conservancy/Brazilian Science Team
#### v1

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [ ]:
#Import geojson extracted from GEE export table task manager
suzano1 = gpd.read_file(r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02h_dados_scripts\source\drive-download-20220328T163524Z-001\NDVI_S2_TOA_2016_2021_suzano_2000.geojson")
suzano2 = gpd.read_file(r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02h_dados_scripts\source\drive-download-20220328T163524Z-001\NDVI_S2_TOA_2016_2021_suzano_4000.geojson")
suzano3 = gpd.read_file(r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02h_dados_scripts\source\drive-download-20220328T163524Z-001\NDVI_S2_TOA_2016_2021_suzano_6000.geojson")
suzano4 = gpd.read_file(r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02h_dados_scripts\source\drive-download-20220328T163524Z-001\NDVI_S2_TOA_2016_2021_suzano_8000.geojson")
suzano5 = gpd.read_file(r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02h_dados_scripts\source\drive-download-20220328T163524Z-001\NDVI_S2_TOA_2016_2021_suzano_10000.geojson")
suzano6 = gpd.read_file(r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02h_dados_scripts\source\drive-download-20220328T163524Z-001\NDVI_S2_TOA_2016_2021_suzano_12000.geojson")

In [ ]:
#Pandas concat geojson
suzano_gdf = pd.concat([suzano1, suzano2, suzano3, suzano4, suzano5, suzano6])

In [ ]:
#Read Geopandas to check columns and rows
suzano_gdf.head()

In [ ]:
#count unique id column
suzano_gdf['id'].nunique()

In [ ]:
#Check columns names
suzano_gdf.columns

In [ ]:
#Check unique METODO column
suzano_gdf['METODO'].unique()

In [ ]:
#Drop PE variable from METODO column
suzano_gdf = suzano_gdf.loc[suzano_gdf['METODO'].isin(['CE', 'CO', 'PC', 'PL', 'RE'])]

In [ ]:
#Transform ANOIMPL column to int
suzano_gdf['ANOIMPL'] = suzano_gdf['ANOIMPL'].astype('int64')
#Select ANOIMPL higher or equal to 2016
suzano_gdf = suzano_gdf[suzano_gdf['ANOIMPL'] >= 2016]

In [ ]:
#Classify forest age in two classes - 0-3 years and 3-6 years
suzano_gdf['idade'] = 'sem valor'

suzano_gdf['idade'] = np.where(suzano_gdf['ANOIMPL'] == 2021, 
                                                 '0-3 anos', suzano_gdf['idade'])
suzano_gdf['idade'] = np.where(suzano_gdf['ANOIMPL'] == 2020, 
                                                 '0-3 anos', suzano_gdf['idade'])
suzano_gdf['idade'] = np.where(suzano_gdf['ANOIMPL'] == 2019, 
                                                 '0-3 anos', suzano_gdf['idade'])
suzano_gdf['idade'] = np.where(suzano_gdf['ANOIMPL'] == 2018, 
                                                 '3-6 anos', suzano_gdf['idade'])
suzano_gdf['idade'] = np.where(suzano_gdf['ANOIMPL'] == 2017, 
                                                 '3-6 anos', suzano_gdf['idade'])
suzano_gdf['idade'] = np.where(suzano_gdf['ANOIMPL'] == 2016, 
                                                 '3-6 anos', suzano_gdf['idade'])

In [ ]:
#Check new column filling
suzano_gdf.head()

In [ ]:
#count 0-3y class by METODO column
suzano_gdf[suzano_gdf['idade'] == '0-3 anos'].groupby(['METODO']).size()

In [ ]:
#count 3-6y class by METODO column
suzano_gdf[suzano_gdf['idade'] == '3-6 anos'].groupby(['METODO']).size()

In [ ]:
#Important to export to shapefile (integrity)
#This part of code eliminate linestrings and other from Geometrycollections

gdf_filtered = suzano_gdf.copy()

for i, row in suzano_gdf.iterrows():
    if type(row.geometry) == shapely.geometry.collection.GeometryCollection:

        # get the polygon and only keep the polygon 
        for shape in row.geometry:
            if type(shape) == shapely.geometry.polygon.Polygon:
                gdf_filtered.at[i, 'geometry'] = shape
                break
#gdf_filtered is the output
                
#suzano_gdf.to_file("suzano_gdf_2016_2021_subset2016.geojson", ddriver='GeoJSON')
#suzano_gdf.to_file("suzano_gdf_2016_2021_subset2016.gpkg",driver="GPKG")

In [ ]:
#Export to shapefile
gdf_filtered.to_file("suzano_gdf_2016_2021_subset2016.shp")

In [ ]:
#Plot test with 10 rows
subset_suzano_gdf = suzano_gdf[:10]

In [ ]:
#Set CODTALHAO Column as index
subset_suzano_gdf.set_index('CODTALHAO',inplace=True)
#subset_suzano_gdf.iloc[:, 4]#.T.plot(grid = True)

In [ ]:
#Test iloc subset by columns position
subset_suzano_gdf.iloc[:,3:16]

In [ ]:
#subset timeseries columns to plot in graph
subset_suzano_gdf_s = subset_suzano_gdf.iloc[:,3:15]#.plot.line()
fig, ax = plt.subplots(figsize=(15,7), dpi=100)
tickvalues = range(0,len(subset_suzano_gdf_s.columns))
ax = subset_suzano_gdf_s.T.plot.line(ax=ax)
plt.title('NDVI plots por talhões 2016-2021 semestral - Suzano',fontsize=10)
plt.xticks(ticks = tickvalues, labels = subset_suzano_gdf_s.columns.tolist(), rotation=45)
plt.show()

In [ ]:
#Read forest and pasture samples (reference) as Geopandas dataframes
amostras_pastagem_NDVI = gpd.read_file(r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02h_dados_scripts\source\drive-download-20220328T163524Z-001\amostras_pastagem_suzano_2016_2021.geojson")
amostras_floresta_NDVI = gpd.read_file(r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02h_dados_scripts\source\drive-download-20220328T163524Z-001\amostras_floresta_suzano_2016_2021.geojson")

In [ ]:
#Set ID as index and drop geometry column
amostras_pastagem_NDVI.set_index('id',inplace=True)
amostras_floresta_NDVI.set_index('id',inplace=True)
amostras_pastagem_NDVI = amostras_pastagem_NDVI.drop(columns=['geometry'])
amostras_floresta_NDVI = amostras_floresta_NDVI.drop(columns=['geometry'])

In [ ]:
#plot pasture NDVI plots
fig, ax = plt.subplots(figsize=(15,7), dpi=100)
tickvalues = range(0,len(amostras_pastagem_NDVI.columns))
ax = amostras_pastagem_NDVI.T.plot.line(ax=ax, legend=False)
plt.title('NDVI amostras pastagem 2016-2021 semestral - Suzano',fontsize=10)
plt.xticks(ticks = tickvalues, labels = amostras_pastagem_NDVI.columns.tolist(), rotation=60)
ax.set_ylim(0.0, 1.0)
plt.savefig('amostras_pastagem_2016_2021_NDVI_S2_TOA.png', dpi=300)
plt.show()

In [ ]:
#plot forest NDVI plots
fig, ax = plt.subplots(figsize=(15,7), dpi=100)
tickvalues = range(0,len(amostras_floresta_NDVI.columns))
ax = amostras_floresta_NDVI.T.plot.line(ax=ax, legend=False)
plt.title('NDVI amostras floresta 2016-2021 semestral - Suzano',fontsize=10)
plt.xticks(ticks = tickvalues, labels = amostras_floresta_NDVI.columns.tolist(), rotation=60)
ax.set_ylim(0.0, 1.0)
plt.savefig('amostras_floresta_2016_2021_NDVI_S2_TOA.png', dpi=300)
plt.show()